# AI Song Writer GPT2



---



#### Import potrzebnych bibliotek

In [ ]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

  Preparing metadata (setup.py) ... done


#### Pobranie **"surowego"** modelu GPT2 124M

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 4.65Git/s]                                                     
Fetching encoder.json: 1.05Mit [00:00, 1.76Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 4.58Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:16, 30.1Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 1.22Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.15Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.08Mit/s]


#### Import wcześniej zescrapowanych danych z pliku .csv, oraz usunięcie rekordów zawierających puste wartości w kolumnie **English** (kolumna która zostanie wykorzystana do operacji dostrojenia modelu)

In [ ]:
import pandas as pd
dataset = pd.read_csv("/content/Merged.csv", index_col = None)

dataset = dataset[dataset['English'].notna()]

#### Oczyszczenie zbioru danych ze znaków, które uznaliśmy za niepotrzebne w trenowaniu modelu, do zbioru tych znaków należą:

*   *
*   ^
*   /



In [ ]:
import re
def clearing(x, column):

  for index, row in x.iterrows():
    row[column] = re.sub(r'[*^=/]', '', row[column])

  return x

clear_data = clearing(dataset, "English")

#### Zapisanie oczyszczonych danych do pliku o nazwie songs z rozszerzeniem .txt, dane z tego pliku bezpośrednio posłużą do treningu modelu.

In [ ]:
filename = "songs.txt"

with open(filename, 'w') as file:
    clear_data['English'].to_csv(file, header=False, index=False)

#### Połączenie naszego Google Drive z środowiskiem wykonawczym w Google Colab

In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


#### Inicjalizacja środowiska Tensorflow, oraz dobór parametrów wykorzystywanych do operacji Fine Tune

*   dataset - plik .txt, z którego dane są wykorzystywane do trenowania modelu GPT2
*   steps - liczba epok, które bedą służyć do treningu
*   restore_from - wartość "fresh" służy do trenowania modelu na nowo dla naszych danych, natomiast "latest" wykorzystujemy do wczytania modelu z ostatniego checkpoint'a
*   run_name - nazwa podfolderu wewnątrz folderu checkpoint, gdzie będzie zapisany model
*   print_every - liczba określająca co ile epok zostaniemy informowani o postępie uczenia
*   sample_every - liczba określająca co ile epok model zwróci nam przykładowe wygenerowane dane
*   save_every - liczba epok, co ile model zostaje zapisany do checkpoint'a

#### Po zakończeniu Fine Tune zapisujemy model w Google Drive

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=filename,
              model_name='124M',
              steps=8500,
              restore_from='fresh',
              run_name='run1',
              print_every=500,
              sample_every=1000,
              save_every=5000,
              )
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

#### Ten zestaw poleceń należy wykonać, jeśli chcemy użyć już gotowego modelu do zadania generowania piosenek, należy pamiętać o poprzednim **zainstalowaniu** odpowiednich bibliotek

In [ ]:
gpt2.mount_gdrive()
gpt2.copy_checkpoint_from_gdrive(run_name='run1')
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading checkpoint checkpoint/run1/model-8500


#### Określenie początkowych wyrazów, od których ma zaczynać się piosenka

In [ ]:
prefix = "Love is"

#### Parametry wykorzystane podczas generowania tekstu

*   length - liczba tokenów z których ma składać się jeden rekord
*   temperature - entropia wyrazów wykorzystanych w nowym utworze (im większa wartość tym mniej standardowych słów będzie się pojawiać)
*   prefix - dane wejściowe
*   nsamples - liczba zwracanych rekordów danych
*   return_as_list - parametr pozwalający na przekazanie wygenerowanego teksu do zmiennej jako listy


#### Przykładowy tekst wygenerowany przez model


In [ ]:
gpt2.generate(sess,
              length=100,
              temperature=0.7,
              prefix=prefix,
              nsamples=5,
            # return_as_list = True
              )

Love is the past, where we can be in the future

[Bridge]
We've been friends for so long
And I'm not even sure I'm okay
I'm a fool
I don't know what it is
But I guess I don't mind

[Outro]
We'll fly out to London
Where we can be in the future
And we'll see
And we'll see
And we'll see
And we'll see
And we'll see
Love is in your heart
And you hear it in your heart,
And I hear it in your heart,

And I write my love down
And I can read your heart-wrenching face on the cover of a magazine
And I can finally see that your eyes are now filled with tears,
I can finally see that you're my light

And I write my love down
And I can read your heart-wrenching face on the cover of a magazine
And
Love is possessed by the devil;
The devil gives him a piece of silverware
So that he can finally feel
that he's not just himself,
that he is all the universe
And everything else in life is nothing
Because everything else in life is nothing"
"You're sitting by yourself
The world chokes you
The thoughts are rus